# 2 Variable Selection via Forward and Backward Search

In [43]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression,ElasticNet,Lasso
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from sklearn.linear_model import RidgeCV,Ridge, RidgeClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Lars

Load the dataset regression.npy, the dataset consists of over 100 predictors. We generated the regression dataset such that only a few predictors are relevant.

In [44]:
with open('regression.npy', 'rb') as f:
    X = np.load(f)
    y = np.load(f)

In [45]:
print("\nFeaures :\n",X)
print("\n Target Variable:\n",y)


Feaures :
 [[0.19151945 0.62210877 0.43772774 ... 0.81920207 0.05711564 0.66942174]
 [0.76711663 0.70811536 0.79686718 ... 0.10310444 0.80237418 0.94555324]
 [0.97903882 0.88123225 0.62768192 ... 0.82215979 0.62796507 0.11792306]
 ...
 [0.94973721 0.43577786 0.01413315 ... 0.22343209 0.98627748 0.52625881]
 [0.50836    0.39705168 0.6062449  ... 0.93781016 0.14590475 0.35087649]
 [0.76700907 0.92954794 0.70505701 ... 0.52896505 0.76347965 0.07979716]]

 Target Variable:
 [15.48730667 22.07740941 17.43403019 ... 24.33341646 19.47937698
 21.73607375]


#### Perform the following experiments using the least angle regression algorithm

In [46]:
print("\nShape of The Covariates are or Dataset without the target variable: \t",X.shape)
print("\nShape of the target variable: \t",y.shape)


Shape of The Covariates are or Dataset without the target variable: 	 (10000, 100)

Shape of the target variable: 	 (10000,)


Adding a ones column to the dataset for the bias term.
Splitting the data into 80:20 ratio

In [47]:
n, m = X.shape
split = int(0.8 * n)
p = np.random.permutation(n)
ones = np.ones(shape=X.shape[0]).reshape(-1, 1)
x = np.concatenate((ones,X), 1)
# Normalizing
from sklearn import preprocessing
x_data = preprocessing.normalize([x])
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y, test_size=0.2,shuffle=True)
print(" X_Train Shape : ",x_train.shape,"\nX_Valid Shape : ",x_valid.shape,"\nY_Train Shape : ",y_train.shape,"\nY_valid Shape : ",y_valid.shape)


 X_Train Shape :  (8000, 101) 
X_Valid Shape :  (2000, 101) 
Y_Train Shape :  (8000,) 
Y_valid Shape :  (2000,)


5. Get the indices of the top k coefficient of these three models and compare them with the features selected via the forward and back search method. Feel free to try different values of k

Finding the top k coefficients and their respective indices(columns)

In [37]:
def Top_k_coeff_indices(arr, k):
    # Finding the indices of the coeffcients in the array after partitioning
    col_indices = np.argpartition(arr, k)[-k:]
    # Getting the top k values 
    Top_k_coeff_values = arr[col_indices]
    # Getting the positions of the top k coefficients from the array
    Top_k_coeff_indices_positions = np.argsort(arr[col_indices])[::-1]
    # Sorting the array of top k coefficients in descending order
    sorted_Top_k_coeff_indices_values = np.sort(Top_k_coeff_values)
    sorted_top_k_coeff_values = sorted_Top_k_coeff_indices_values[::-1]
    print("\n The maximum Coefficient values: \n")
    print(sorted_top_k_coeff_values)
    print("\n The Indices of the maximum Coefficient values: \n")
    print(Top_k_coeff_indices_positions)
    # return sorted_top_k_coeff_values, Top_k_coeff_indices_positions

## 1. Forward Search

### FORWARD SEARCH IMPLEMENTATION FROM SCRATCH

In [38]:
def forward_search(model, x_train, y_train, x_valid, y_valid):
    N,M = x_train.shape
    print(f"Total Number of Features:{M}")
    covariates_list = set(range(M))
    features = set()
    initial_error = 1e10
    best_error = 1e10
    best_feature = 1
    while best_feature is not None:
        print("Best MSE:", best_error, "\tFinal Features:", features)
        best_feature = None
        best_error = initial_error
        for f in covariates_list - features:
            # Concatenate Covariates columns
            add_new_feature = list(features | {f})
            # Train model till the concatenated covariates
            model.fit(x_train[:, add_new_feature], y_train)
            # Predict y_hat for validation set till the concatenated covariates
            y_hat= model.predict(x_valid[:, add_new_feature])
            # Compute the error for verification if the covariate is significant for the model or not
            error_computed = mse(y_valid, y_hat)
            if error_computed < best_error:
                best_feature = f
                best_error = error_computed
        if best_error < initial_error:
            features = features | {best_feature}
            print("Important features obtained: ",features)
            initial_error = best_error
    print(" The top 5 coefficients along with the idices")
    return Top_k_coeff_indices(model.coef_,5)
    # return features

In [11]:
model = linear_model.Lars()
forward_search( model ,x_train , y_train , x_valid , y_valid )

Total Number of Features:101
Best MSE: 10000000000.0 	Final Features: set()
Important features obtained:  {4}
Best MSE: 15.83745742162434 	Final Features: {4}
Important features obtained:  {1, 4}
Best MSE: 11.835015836818465 	Final Features: {1, 4}
Important features obtained:  {1, 2, 4}
Best MSE: 8.36812921598756 	Final Features: {1, 2, 4}
Important features obtained:  {1, 2, 4, 5}
Best MSE: 5.86230330917619 	Final Features: {1, 2, 4, 5}
Important features obtained:  {1, 2, 4, 5, 56}
Best MSE: 5.8555060911605175 	Final Features: {1, 2, 4, 5, 56}
Important features obtained:  {1, 2, 4, 5, 36, 56}
Best MSE: 5.850436438271824 	Final Features: {1, 2, 4, 5, 36, 56}
Important features obtained:  {1, 2, 4, 5, 36, 56, 91}
Best MSE: 5.845381730293763 	Final Features: {1, 2, 4, 5, 36, 56, 91}
Important features obtained:  {1, 2, 4, 5, 36, 56, 91, 63}
Best MSE: 5.840286899663979 	Final Features: {1, 2, 4, 5, 36, 56, 91, 63}
Important features obtained:  {1, 2, 4, 5, 36, 56, 91, 31, 63}
Best MSE:

### FORWARD SEARCH IMPLEMENTATION WITH SKLEARN

In [39]:
feature_selection_model = SequentialFeatureSelector(model, n_features_to_select= 50, direction="forward")
feature_selection_model.fit(X, y)
features_boolean = feature_selection_model.get_support().tolist()
feature_list = []
for i in range(len(features_boolean)):
    if features_boolean[i]:
        feature_list.append(i)

print("\nBest Features obtained from Forward Search :\n", feature_list)
# Training our data with the new set of features obtained from Forward Search
lars_new_model = Lars()
lars_new_model.fit(x_train[:,feature_list],y_train)
y_hat = lars_new_model.predict(x_valid[:,feature_list])
MSE_score = mean_squared_error(y_valid,y_hat)
print("\nMean Squarred Error On The Features Obtained from Forward Search:\t ",MSE_score)


Best Features obtained from Forward Search :
 [0, 1, 2, 3, 4, 6, 7, 12, 14, 17, 21, 22, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 40, 41, 42, 46, 47, 49, 50, 51, 54, 55, 60, 61, 62, 65, 69, 72, 74, 76, 78, 79, 80, 81, 84, 86, 90, 96, 97]

Mean Squarred Error On The Features Obtained from Forward Search:	  8.223846520097563


In [41]:
## TOP 5 Coefficients along with their indicies
Top_k_coeff_indices(lars_new_model.coef_,5)


 The maximum Coefficient values: 

[6.64179097 0.1956206  0.16666649 0.01025202 0.00666463]

 The Indices of the maximum Coefficient values: 

[3 2 1 0 4]


## 2. Backward Search

### BACKWARD SEARCH IMPLEMENTATION WITH SKLEARN

In [48]:
feature_selection_model = SequentialFeatureSelector(model, n_features_to_select= 50, direction="backward")
feature_selection_model.fit(X, y)
features_boolean = feature_selection_model.get_support().tolist()
feature_list = []
for i in range(len(features_boolean)):
    if features_boolean[i]:
        feature_list.append(i)
print("\nBest Features obtained from Backward Search :\n", feature_list)
# Training our data with the new set of features obtained from Backward Search
lars_new_model = Lars()
lars_new_model.fit(x_train[:,feature_list],y_train)
y_hat = lars_new_model.predict(x_valid[:,feature_list])
MSE_score = mean_squared_error(y_valid,y_hat)
print("\nMean Squarred Error On The Features Obtained from Backward Search:\t ",MSE_score)


Best Features obtained from Backward Search :
 [0, 1, 2, 3, 4, 6, 7, 12, 14, 17, 21, 22, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 40, 41, 42, 46, 47, 49, 50, 51, 54, 55, 60, 61, 62, 65, 69, 72, 74, 76, 78, 79, 80, 81, 84, 86, 90, 96, 97]

Mean Squarred Error On The Features Obtained from Backward Search:	  8.645598420338786


In [49]:
## TOP 5 Coefficients along with their indicies
Top_k_coeff_indices(lars_new_model.coef_,5)


 The maximum Coefficient values: 

[6.77221487 0.12951897 0.12025592 0.0783876  0.01658407]

 The Indices of the maximum Coefficient values: 

[3 1 2 4 0]


### BACKWARD SEARCH IMPLEMENTATION FROM SCRATCH

In [12]:
def backward_search(model, x_train, y_train, x_valid, y_valid):
    N,M = x_train.shape
    print(f"Total Number of Features:{M}")
    covariates_list = set(range(M))
    best_feature = 1
    model.fit(x_train, y_train)
    pred= model.predict(x_valid)
    error = mse(y_valid, pred)
    while best_feature is not None:
        print("MSE(validation):", error, "    Features:", covariates_list)
        best_feature = None
        best_error = error
        for f in covariates_list:
            add_new_feature = list(covariates_list-{f})
            model.fit(x_train[:, :len(add_new_feature)], y_train)
            pred = model.predict(x_valid[:, :len(add_new_feature)])
            error_computed = mse(y_valid, pred)
            if error_computed < best_error:
                best_feature = f
                best_error = error_computed
        if best_error < error:
            covariates_list = covariates_list - {best_feature}
            print("Important features obtained: ",covariates_list)
            error = best_error
    print(" The top 5 coefficients along with the idices")
    return Top_k_coeff_indices(model.coef_,5)
    # return covariates_list

In [13]:
model = linear_model.Lars()
backward_search( model ,x_train , y_train , x_valid , y_valid )

Total Number of Features:101
MSE(validation): 5.92812398907075     Features: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}
Important features obtained:  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100}
MSE(validation): 5.928123074225292     Features: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,

# 3 Regularization

Variable selection via forward and backward search drops some predictors; in some cases, we don’t want to remove these predictors. Rather we want their coefficients to be small as possible. We are going to test the effect of the regularization term alpha. 
### Try the following alpha values: [10, 1, 0.1, 0.0001, 0.00001 ], use regression.npy dataset

In [15]:
alphas = {'alpha':[10, 1, 0.1, 0.0001, 0.00001]}

## I) Grid Search
### a)Ridge

In [16]:
# define the model/ estimator
model = Ridge()

# define the grid search
Ridge_reg= GridSearchCV(model, alphas,scoring='neg_mean_squared_error',cv=5)
#fit the grid search
Ridge_reg.fit(x_train,y_train)
# best estimator
print("\n The Best Hyperparameter for Ridge Regression from the given list of alpha values is : \n")
print(Ridge_reg.best_params_)

# Fitting the model with the best hyperparameter obtained
model = Ridge(alpha = Ridge_reg.best_params_['alpha'])
model.fit(x_train, y_train)
yhat = model.predict(x_valid)
mse = mean_squared_error(y_valid, yhat)
print("The Mean Squarred Error for Ridge Regression is :\t ",mse)

Top_k_coeff_indices(model.coef_,5)


 The Best Hyperparameter for Ridge Regression from the given list of alpha values is : 

{'alpha': 1}
The Mean Squarred Error for Ridge Regression is :	  5.928912324660127

 The maximum Coefficient values: 

[6.69635381e+00 1.06661918e-01 8.84709707e-02 1.89214737e-02
 5.48353351e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]


### b) Lasso

In [17]:

# define the model/ estimator
model = Lasso()

# define the grid search
lasso= GridSearchCV(model, alphas,cv=5)
#fit the grid search
lasso.fit(x_train,y_train)
# best estimator
print("\n The Best Hyperparameter for Lasso Regression from the given list of alpha values is : \n")
print(lasso.best_params_)
model = Lasso(alpha = lasso.best_params_['alpha'])
model.fit(x_train, y_train)
yhat = model.predict(x_valid)
mse1 = mean_squared_error(y_valid, yhat)
print("The Mean Squarred Error for Lasso Regression is :\t ",mse1)

Top_k_coeff_indices(model.coef_,5)


 The Best Hyperparameter for Lasso Regression from the given list of alpha values is : 

{'alpha': 0.0001}
The Mean Squarred Error for Lasso Regression is :	  5.927321185786832

 The maximum Coefficient values: 

[6.70514057e+00 1.05630210e-01 8.73115691e-02 1.75780490e-02
 4.43979327e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]


### c) Elastic

In [18]:

# define the model/ estimator
model = ElasticNet()

# define the grid search
elasticnet= GridSearchCV(model, alphas,cv=5)
#fit the grid search
elasticnet.fit(x_train,y_train)
# best estimator
print("\n The Best Hyperparameter for ElasticNet Regression from the given list of alpha values is : \n")
print(elasticnet.best_params_)
model = ElasticNet(alpha = elasticnet.best_params_['alpha'])
model.fit(x_train, y_train)
yhat = model.predict(x_valid)
mse2 = mean_squared_error(y_valid, yhat)
print("The Mean Squarred Error for ElasticNet Regression is :\t ",mse2)

Top_k_coeff_indices(model.coef_,5)


 The Best Hyperparameter for ElasticNet Regression from the given list of alpha values is : 

{'alpha': 0.0001}
The Mean Squarred Error for ElasticNet Regression is :	  5.92802582780389

 The maximum Coefficient values: 

[6.70173765e+00 1.06150351e-01 8.78922987e-02 1.82395330e-02
 4.98138045e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]


## II) Random Search

### i) Ridge

In [19]:

# define the model/ estimator
model = Ridge()

# define the grid search
Ridge_reg= RandomizedSearchCV(model, alphas,scoring='neg_mean_squared_error',cv=5)
#fit the grid search
Ridge_reg.fit(x_train,y_train)
# best estimator
print("\n The Best Hyperparameter for Ridge Regression from the given list of alpha values is : \n")
print(Ridge_reg.best_params_)

model = Ridge(alpha = Ridge_reg.best_params_['alpha'])
model.fit(x_train, y_train)
yhat = model.predict(x_valid)
mse3 = mean_squared_error(y_valid, yhat)
print("The Mean Squarred Error for Ridge Regression is :\t ",mse3)

Top_k_coeff_indices(model.coef_,5)



 The Best Hyperparameter for Ridge Regression from the given list of alpha values is : 

{'alpha': 1}
The Mean Squarred Error for Ridge Regression is :	  5.928912324660127

 The maximum Coefficient values: 

[6.69635381e+00 1.06661918e-01 8.84709707e-02 1.89214737e-02
 5.48353351e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]


### ii) Lasso

In [20]:
# define the model/ estimator
model = Lasso()

# define the grid search
lasso= RandomizedSearchCV(model, alphas,cv=5)
#fit the grid search
lasso.fit(x_train,y_train)
# best estimator
print("\n The Best Hyperparameter for Lasso Regression from the given list of alpha values is : \n")
print(lasso.best_params_)
model = Lasso(alpha = lasso.best_params_['alpha'])
model.fit(x_train, y_train)
yhat = model.predict(x_valid)
mse4 = mean_squared_error(y_valid, yhat)
print("The Mean Squarred Error for Ridge Regression is :\t ",mse4)

Top_k_coeff_indices(model.coef_,5)


 The Best Hyperparameter for Lasso Regression from the given list of alpha values is : 

{'alpha': 0.0001}
The Mean Squarred Error for Ridge Regression is :	  5.927321185786832

 The maximum Coefficient values: 

[6.70514057e+00 1.05630210e-01 8.73115691e-02 1.75780490e-02
 4.43979327e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]


### iii) Elastic Net

In [21]:
from sklearn.linear_model import ElasticNet
# define the model/ estimator
model = ElasticNet()

# define the grid search
elasticnet= RandomizedSearchCV(model, alphas,cv=5)
#fit the grid search
elasticnet.fit(x_train,y_train)
# best estimator
print("\n The Best Hyperparameter for Lasso Regression from the given list of alpha values is : \n")
print(elasticnet.best_params_)
model = ElasticNet(alpha = elasticnet.best_params_['alpha'])
model.fit(x_train, y_train)
yhat = model.predict(x_valid)
mse5 = mean_squared_error(y_valid, yhat)
print("The Mean Squarred Error for Ridge Regression is :\t ",mse5)

Top_k_coeff_indices(model.coef_,5)


 The Best Hyperparameter for Lasso Regression from the given list of alpha values is : 

{'alpha': 0.0001}
The Mean Squarred Error for Ridge Regression is :	  5.92802582780389

 The maximum Coefficient values: 

[6.70173765e+00 1.06150351e-01 8.78922987e-02 1.82395330e-02
 4.98138045e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]


In [22]:
print("\n All the mses for the 6 models are as follows:\n")
print("\nMSE1: ",mse,"\n")
print("\nMSE2: ",mse1,"\n")
print("\nMSE3: ",mse2,"\n")
print("\nMSE4: ",mse3,"\n")
print("\nMSE5: ",mse4,"\n")
print("\nMSE6: ",mse5,"\n")


 All the mses for the 6 models are as follows:


MSE1:  5.928912324660127 


MSE2:  5.927321185786832 


MSE3:  5.92802582780389 


MSE4:  5.928912324660127 


MSE5:  5.927321185786832 


MSE6:  5.92802582780389 



3. Briefly discuss the effect of high and low values of alpha. Then, return the best three models with their respective alpha values using the appropriate metric

In [23]:
print("\n Best 3 models are as follows:\n")
print("\nMSE1: ",mse,"\n")
print("\nMSE2: ",mse1,"\n")
print("\nMSE3: ",mse2,"\n")


 Best 3 models are as follows:


MSE1:  5.928912324660127 


MSE2:  5.927321185786832 


MSE3:  5.92802582780389 



Ridge or Lasso regression is basically Shrinkage(regularization) techniques, which uses different parameters and values to shrink or penalize the coefficients.When we fit a model, we are asking it to learn a set of coefficients that best fit over the training distribution as well as hope to generalize on test data points as well. Learning those coefficients can be done in various ways and multiple techniques are there to reduce the error in coefficients such as LMS(Least mean Squared), RSS(Residual Sum of Squares).

For Ridge Regression ⍺ given is a parameter in the ridge function. So, by changing the values of alpha, we are controlling the penalty term. The higher the values of alpha, the bigger ⍺ is the penalty and therefore the magnitude of coefficients is reduced.

For Lasso Regression the increasing value of the regularization parameter means increasing regularization strength, the absolute values of weights would need to decrease (shrink) to keep the overall value of the loss function minimized. The optimization of the Lasso loss function results in some of the weights becoming zero and hence can be seen as a method of selection of the features.

4. Compare the best three models to the models from question 2. What do you observe?

Since both Random Search and Grid Search gives us the same value for Mean Squarred Error in all the three cases that is Ridge, Lasso and ElasticNet, therefore, I have just taken the 3 models from Grid Search for comparison with the model in question 2.

We observe that the MSE obtained after performing forward search is more compared to the MSEs obtained from Grid Search for Ridge, Lasso and Elastic net. The MSEs for grid search are printed above and the MSE obtained in forward search is 8.223846520097563 which is more compared to the others.

We also observe that MSE obtained after performing the backward search is also more compared to the MSEs obtained from Grid Search for Ridge, Lasso and Elasticnet. The MSE for backward search is observed as 8.645598420338786.

5. Get the indices of the top k coefficient of these three models and compare them with the features selected via the forward and back search method. Feel free to try different values of k

#### For Forward Search obtained from above:

The maximum Coefficient values: 

[10.07086953  6.65138633  5.06005849  0.12167152  0.02895172]

 The Indices of the maximum Coefficient values: 

[2 4 1 3 0]

 #### For Ridge Regression using GridSearch obtained from above::
 The maximum Coefficient values: 

[6.69635381e+00 1.06661918e-01 8.84709707e-02 1.89214737e-02
 5.48353351e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]

 #### For Lasso Regression using Grid Search obtained from above:
 The maximum Coefficient values: 

[6.70514057e+00 1.05630210e-01 8.73115691e-02 1.75780490e-02
 4.43979327e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]

#### For ElasticNet Regression using Grid Search obtained from above:
The maximum Coefficient values: 

[6.70173765e+00 1.06150351e-01 8.78922987e-02 1.82395330e-02
 4.98138045e-03]

 The Indices of the maximum Coefficient values: 

[2 0 1 4 3]

We can see that the indices obtained for top 5 coefficients from Forward Search is the same but the coefficient values are different from the ones obtained from Grid Search Algorithm for all Ridge, Lasso and Elastic Net.